In [1]:
#importing libraries
import requests
import pandas as pd
import time

In [2]:
#Keys
API_Key = "AI----------------------------nw"
Channel_ID = "UC----------------------Xg"

In [3]:
#Getting counts of view, like

def get_video_details(video_id):

    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_Key
    response_video_stats = requests.get(url_video_stats).json()
        
    view_count = response_video_stats['items'][0] ['statistics']['viewCount']
    like_count = response_video_stats['items'][0] ['statistics']['likeCount']
        
    return view_count, like_count

In [4]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_Key+"&channelId="+Channel_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
        
        response = requests.get(url).json()
        time.sleep(1)                                              #sleep for 1 second before starting the loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count = get_video_details(video_id)
        
#save data into df
                df = df.append({'video_id': video_id, 'video_title': video_title, 'upload_date': upload_date, 
                        'view_count': view_count,'like_count': like_count},ignore_index = True)
        
        try:
            if response['nextPageToken'] != None: #if none, loop reached the last page of the video and breaks
                pageToken = "pageToken=" + response['nextPageToken']
                
        except:
            break
            
        
    return df
        

In [5]:
#main

#create a new dataframe

df2 = pd.DataFrame(columns=["video_id", "video_title", "upload_date", "view_count","like_count"])

df2 = get_videos(df2)

In [6]:
df2

,video_id,video_title,upload_date,view_count,like_count
0,XRPyj7cKVsQ,Formulas in Excel | Excel Tutorials for Beginners,2021-12-21,2145,136
1,lH7HfwUFnYA,Pivot Tables in Excel | Excel Tutorials for Be...,2021-12-14,9655,446
2,zOR0-nygfDE,Tableau Tutorial for Beginners | Full Project ...,2021-12-07,7086,176
3,A4SVUF-fTwc,Tableau Tutorial for Beginners | Joins | Part 4/5,2021-11-23,5620,172
4,FnTunxd8tAM,Top 5 Major Red Flags in Job Descriptions,2021-11-18,9343,565
...,...,...,...,...,...
117,4rfr6A3lO-Y,Data Analyst Resume | Reviewing My Resume! | F...,2020-01-30,35285,1000
118,OTq2NRy_AGs,Working at a Big Company Vs Small Company | To...,2020-01-25,6628,208
119,ya28cb3zFGE,Data Analyst Salary | 100k with No Experience,2020-01-23,40779,1544
120,Hsi2BG0SOiQ,Truth About Big Companies | Told by a Fortune ...,2020-01-21,4473,175


In [7]:
df2.to_excel('youtube_data.xlsx', index=False)